In [2]:
from numba import jit, cuda

from imageai.Detection import ObjectDetection
import os
import sys
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [3]:
#!/home/natnael_masresha/week11/.venv/bin/python

sys.path.append("../scripts")
from load_data import Loader

loader = Loader()

#!/home/natnael_masresha/week11/.imageai/bin/python


In [4]:
# constants

IMAGES_EXT = ["JPG","PNG","GIF","WEBP","TIFF","PSD","RAW","BMP","HEIF","INDD","JPEG"]

VIDEO_EXT = ["WEBM","MPG","MP2","MPEG","MPE","MPV","OGG","MP4","M4P","M4V","AVI","WMV","MOV","QT","FLV","SWF"]

In [5]:
# @jit(target_backend='cuda') 
def get_files_name(directory:str, filter_extension:list=None,exclude_names:list=None)->list:
    
    # directory = f'/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/{directory}/'
    
    files = []
    for filename in os.listdir(directory):
        f = os.path.join(directory, filename)
        
        # checking if it is a file
        if filter_extension != None:
            if os.path.isfile(f):
                if ~any(substring in filename.split('.')[0] for substring in exclude_names):
                    if filename.split('.')[-1].upper() in filter_extension:
                        files.append(filename)
        else:
            if os.path.isfile(f):
                if ~any(substring in filename.split('.')[0] for substring in exclude_names):
                    files.append(filename)

    return files

In [7]:

detector = ObjectDetection()
execution_path = os.getcwd()
# if model_path == None:
#     model_path = "../junk/data/resnet50_coco_best_v2.1.0.h5"
model_path = "../junk/data/resnet50_coco_best_v2.1.0.h5"

detector.setModelTypeAsRetinaNet()
detector.setModelPath( os.path.join(execution_path , model_path))
detector.loadModel()

def get_model(img_path):
    detections = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , img_path), output_image_path=os.path.join(execution_path , model_path+"newI.jpg"))
    return detections

In [8]:
@jit(target_backend='cuda') 
def get_objects(img_path:str,model_path:str = None,print_objects:bool = False):
    
    detections = get_model(img_path)
    objects = []

    for eachObject in detections:
        objects.append(eachObject["name"])
        if print_objects:
            print(eachObject["name"] , " : " , eachObject["percentage_probability"] )
    return objects

In [9]:
# @jit(target_backend='cuda') 
def get_unique_objects(all_objects:list):
    unique_objects = list(set(all_objects))
    unique_objects_count = len(unique_objects)
    all_objects_count = len(all_objects)
    
    return all_objects,all_objects_count,unique_objects_count

In [10]:
# all in one
# @jit(target_backend='cuda') 
def add_feature(game_id:str):
    path  = f'/home/amanuel_zewdu/creative_image_optimization/data/Challenge_Data/Assets/{game_id}/'
    exclude = ["text","video","background"]
    file_names = get_files_name(path,IMAGES_EXT,exclude)
    all_objects = []
    for file_name in file_names:
        all_text = get_objects(path+file_name)
        all_objects.extend(all_text)
    return get_unique_objects(all_objects)

In [11]:
bucket = "s3://10ac-batch-6/data/w11/Challenge_Data.zip"
file_path = "Challenge_Data/performance_data.csv"

df = loader.load_csv(bucket,file_path)

In [12]:
tst_df = df.sample(2)

In [17]:
df[['all_objects','all_objects_count','unique_objects_count']] = df.game_id.apply(lambda x:pd.Series(add_feature(x)))

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [18]:
df.to_csv('game_id_with_objects.csv',index=False)

In [20]:
# df.head()